In [ ]:
pip install tfx

In [ ]:
import os
import tensorflow as tf

from tfx.components import CsvExampleGen
from tfx.components import ExampleValidator
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Transform
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict

import pprint

pp = pprint.PrettyPrinter()

In [ ]:
from google.colab import drive
drive.mount('./content')

Drive already mounted at ./content; to attempt to forcibly remount, call drive.mount("./content", force_remount=True).


In [ ]:
_pipeline_root = 'metro_traffic_pipeline/'
_data_root = '/content/content/MyDrive/mlops/course2/week2/data2'
_data_filepath = os.path.join(_data_root, 'Metro_Interstate_Traffic_Volume.csv')


In [ ]:
!head {_data_filepath}

holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
None,288.28,0.0,0.0,40,Clouds,scattered clouds,2012-10-02 09:00:00,5545
None,289.36,0.0,0.0,75,Clouds,broken clouds,2012-10-02 10:00:00,4516
None,289.58,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 11:00:00,4767
None,290.13,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 12:00:00,5026
None,291.14,0.0,0.0,75,Clouds,broken clouds,2012-10-02 13:00:00,4918
None,291.72,0.0,0.0,1,Clear,sky is clear,2012-10-02 14:00:00,5181
None,293.17,0.0,0.0,1,Clear,sky is clear,2012-10-02 15:00:00,5584
None,293.86,0.0,0.0,1,Clear,sky is clear,2012-10-02 16:00:00,6015
None,294.14,0.0,0.0,20,Clouds,few clouds,2012-10-02 17:00:00,5791


In [ ]:
context = InteractiveContext(pipeline_root=_pipeline_root)

In [ ]:
example_gen = CsvExampleGen(input_base=_data_root)

In [ ]:
context.run(example_gen)


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 9
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [ ]:
try:
  artifact = example_gen.outputs['examples'].get()[0]
  print(artifact.uri)
  print(artifact.split_names)
except IndexError:
    print("context.run() was no-op")
    examples_path = './metro_traffic_pipeline/CsvExampleGen/examples'
    dir_id = os.listdir(examples_path)[0]
    artifact_uri = f'{examples_path}/{dir_id}'

else:
    artifact_uri = artifact.uri


metro_traffic_pipeline/CsvExampleGen/examples/9
["train", "eval"]


In [ ]:
train_uri = os.path.join(artifact_uri, 'Split-train')
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [ ]:
def get_records(dataset, num_records):

  records = []
  for tfrecord in dataset.take(num_records):

    serialized_example = tfrecord.numpy()
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    example_dict = MessageToDict(example)
    records.append(example)
  return records

In [ ]:
sample_records = get_records(dataset, 1)
pp.pprint(sample_records) 

[features {
  feature {
    key: "clouds_all"
    value {
      int64_list {
        value: 90
      }
    }
  }
  feature {
    key: "date_time"
    value {
      bytes_list {
        value: "2012-10-02 11:00:00"
      }
    }
  }
  feature {
    key: "holiday"
    value {
      bytes_list {
        value: "None"
      }
    }
  }
  feature {
    key: "rain_1h"
    value {
      float_list {
        value: 0.0
      }
    }
  }
  feature {
    key: "snow_1h"
    value {
      float_list {
        value: 0.0
      }
    }
  }
  feature {
    key: "temp"
    value {
      float_list {
        value: 289.5799865722656
      }
    }
  }
  feature {
    key: "traffic_volume"
    value {
      int64_list {
        value: 4767
      }
    }
  }
  feature {
    key: "weather_description"
    value {
      bytes_list {
        value: "overcast clouds"
      }
    }
  }
  feature {
    key: "weather_main"
    value {
      bytes_list {
        value: "Clouds"
      }
    }
  }
}
]


In [ ]:
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])

context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 10
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [ ]:
context.show(statistics_gen.outputs['statistics'])

In [ ]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'])
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 11
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [ ]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'clouds_all',INT,required,,-
'date_time',BYTES,required,,-
'holiday',STRING,required,,'holiday'
'rain_1h',FLOAT,required,,-
'snow_1h',FLOAT,required,,-
'temp',FLOAT,required,,-
'traffic_volume',INT,required,,-
'weather_description',STRING,required,,'weather_description'
'weather_main',STRING,required,,'weather_main'


,Values
Domain,
'holiday',"'Christmas Day', 'Columbus Day', 'Independence Day', 'Labor Day', 'Martin Luther King Jr Day', 'Memorial Day', 'New Years Day', 'None', 'State Fair', 'Thanksgiving Day', 'Veterans Day', 'Washingtons Birthday'"
'weather_description',"'SQUALLS', 'Sky is Clear', 'broken clouds', 'drizzle', 'few clouds', 'fog', 'freezing rain', 'haze', 'heavy intensity drizzle', 'heavy intensity rain', 'heavy snow', 'light intensity drizzle', 'light intensity shower rain', 'light rain', 'light rain and snow', 'light shower snow', 'light snow', 'mist', 'moderate rain', 'overcast clouds', 'proximity shower rain', 'proximity thunderstorm', 'proximity thunderstorm with drizzle', 'proximity thunderstorm with rain', 'scattered clouds', 'shower drizzle', 'shower snow', 'sky is clear', 'sleet', 'smoke', 'snow', 'thunderstorm', 'thunderstorm with drizzle', 'thunderstorm with heavy rain', 'thunderstorm with light drizzle', 'thunderstorm with light rain', 'thunderstorm with rain', 'very heavy rain'"
'weather_main',"'Clear', 'Clouds', 'Drizzle', 'Fog', 'Haze', 'Mist', 'Rain', 'Smoke', 'Snow', 'Squall', 'Thunderstorm'"


In [ ]:
example_validator = ExampleValidator(statistics=statistics_gen.outputs['statistics'], schema=schema_gen.outputs['schema'])

In [ ]:
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 12
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [ ]:
context.show(example_validator.outputs['anomalies'])

In [ ]:
_traffic_constant_module_file = 'traffic_constants.py'

In [ ]:
%%writefile {_traffic_constant_module_file}

DENSE_FLOAT_FEATURE_KEYS = ['temp', 'snow_1h']
BUCKET_FEATURE_KEYS = ['rain_1h']
FEATURE_BUCKET_COUNT = {'rain_1h': 3}
RANGE_FEATURE_KEYS = ['clouds_all']
RANGE_FEATURE_KEYS
VOCAB_SIZE = 1000
OOV_SIZE = 10

VOCAB_FEATURE_KEYS = [
  'holiday',
  'weather_main',
  'weather_description'
]

CATEGORICAL_FEATURE_KEYS = ['hour', 'day', 'day_of_week', 'month']

VOLUME_KEY = 'traffic_volume'

def transformed_name(key):
  return key + '_xf'

Overwriting traffic_constants.py


In [ ]:
_traffic_transform_module_file = 'traffic_transform.py'

In [ ]:
%%writefile {_traffic_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import traffic_constants

_DENSE_FLOAT_FEATURE_KEYS = traffic_constants.DENSE_FLOAT_FEATURE_KEYS
_RANGE_FEATURE_KEYS = traffic_constants.RANGE_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = traffic_constants.VOCAB_FEATURE_KEYS
_VOCAB_SIZE = traffic_constants.VOCAB_SIZE
_OOV_SIZE = traffic_constants.OOV_SIZE
_CATEGORICAL_FEATURE_KEYS = traffic_constants.CATEGORICAL_FEATURE_KEYS
_BUCKET_FEATURE_KEYS = traffic_constants.BUCKET_FEATURE_KEYS
_FEATURE_BUCKET_COUNT = traffic_constants.FEATURE_BUCKET_COUNT
_VOLUME_KEY = traffic_constants.VOLUME_KEY
_transformed_name = traffic_constants.transformed_name

def preprocessing_fn(inputs):
  outputs = {}

  for key in _DENSE_FLOAT_FEATURE_KEYS:
    outputs[_transformed_name(key)] = tft.scale_to_z_score(inputs[key])
  
  for key in _RANGE_FEATURE_KEYS:
    outputs[_transformed_name(key)] = tft.scale_to_z_score(inputs[key])

  for key in _VOCAB_FEATURE_KEYS:
    outputs[_transformed_name(key)] = tft.compute_and_apply_vocabulary(
            inputs[key], 
            top_k=_VOCAB_SIZE, 
            num_oov_buckets=_OOV_SIZE)


  for key in _CATEGORICAL_FEATURE_KEYS:
    outputs[_transformed_name(key)] =  inputs[key]

  traffic_volume = tf.cast(_fill_in_missing(inputs[_VOLUME_KEY]), tf.float32)
                  tf.cast(_fill_in_missing(inputs[_VOLUME_KEY]), tf.float32)
  
  outputs[_transformed_name(_VOLUME_KEY)] = tf.cast(
    tf.greater(traffic_volume, 
    tft.mean(tf.cast(traffic_volume,
    tf.float32))), tf,int64
  )
  return outputs

def _fill_in_missing(x):

  default_value = '' if x.dtype == tf.string else 0

  return tf.squeeze(
    tf.sparse.to_dense(tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]), 
    default_value),
  axis=1)




Overwriting traffic_transform.py


In [ ]:
sp_input = tf.sparse.SparseTensor(
  dense_shape=[3, 5],
  values=[7, 8, 9],
  indices =[[0, 1],
            [0, 3],
            [2, 0]])

In [ ]:
tf.squeeze(tf.sparse.to_dense(sp_input).numpy()).shape


TensorShape([3, 5])

In [ ]:
tf.get_logger().setLevel('ERROR')

transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_traffic_transform_module_file))
    
    
    

# Run the component
context.run(transform)

KeyError: ignored